# Working with Excel files
Excel is the "Swiss army knife" of business analytics. For those of you who have taken my **MIS 4460/5460 - Business Analytics** course, you know that Excel is extraordinarily powerful and has numerous features and capabilities that the vast majority of users (even those that think they are pretty good with Excel) don't even know exist. 

This notebook is based on the **Chapter 4: Working with Excel Files** of "Data Wrangling with Python" (DWwP). The data files we'll use are *sites_1_6.xlsx* and *SOWC 2014 Stat Tables_Table 9.xlsx*. I've included both in the `/data` subfolder in our Downloads file. You can also go get the second file from the `/data-wrangling-master` archive we downloaded from 
https://github.com/jackiekazil/data-wrangling. I created *sites-1_6.xlsx* from a research project I'm working on.

## Installing Python packages
Okay, first we need to talk about the whole issue of package management. This is a little confusing in general, and even more so when using the Anaconda Python distro. As we'll see when learning R, package management there is a little easier.

### Pretend we aren't using Anaconda for a moment
While Python comes with a ton of libraries built in to the standard library, there are thousands of external libraries or *packages* available for you to use. These packages are collected in an online repository called PyPI (the Python Package Index) at https://pypi.python.org/pypi. Let's go there and get a sense of the layout of the site. 

Let's search for packages related to Excel. There are a bunch. In this tutorial we are going to use three of these packages:

* `xlrd` - Reads Excel files
* `xlwt` - Writes and formats Excel files
* `xlutils` - A set of tools for more advanced operations in Excel (requires `xlrd` and `xlwt`)

**IMPORTANT** - both `xlrd` and `xlwt` come pre-installed in Anaconda. Do NOT reinstall them.

To install packages that live in PyPI, we use another Python package called `pip` (https://pip.pypa.io/en/stable/ and https://en.wikipedia.org/wiki/Pip_(package_manager)). 

To install a Python package from PyPI using `pip` such as `xlutils`, all we have to do is open a shell window and type:

    pip install xlutils
    
A bunch of stuff will scroll down the screen as `pip` figures out all dependencies for the package you want to install and eventually will prompt you for confirmation on continuing. Once a package is successfully installed, you can use it. 

To uninstall:

    pip uninstall xlutils
    
While installing and uninstalling packages with `pip` is pretty easy, finding the right package is a little trickier. There are often multiple packages for doing the same or similar things. And of course, the quality of packages can vary tremendously. It's a giant sea of open source software submitted by a sea of people. I always do a bunch of Googling and reading to figure out which packages seem to get recommended by others and are under active development or at least have long term stability. Let's put "python read Excel files" into Google and see what we find. I can tell you one thing you'll find - there are a lot of really dedicated and helpful Python developers and power users who write all kinds of blogs, tutorials, and forum posts to share their knowledge. One of the great benefits of using open source software is the communities and ecosystems that grow around many software packages (like Python).

One particularly nice tutorial on this topic is https://www.datacamp.com/community/tutorials/python-excel-tutorial.

### But we are using Anaconda Python
While we can use `pip` with Anaconda, it comes with its own package management program called `conda` (http://conda.pydata.org/docs/intro.html). To see all of the packages we have installed, run the following command from the shell:

In [ ]:
!conda list


You'll notice that both `xlrd` and `xlwt` are already installed and `xlutils` is not. If they aren't, perhaps I created a conda virtual env (see below) and didn't include them. To install them (if needed), we use the shell and type:

    conda install xlrd
    conda install xlwt
    
Let's also install `openpyxl'

    conda install openpyxl
    

### More on conda
Conda is used for much more than simply installing and uninstalling packages. As you become more experienced with Python, you'll start to learn about things like using *virtual environments* and even creating and distributing your own packages. Here's the first two paragraphs from the Conda documentation:

> Conda is a package manager application that quickly installs, runs, and updates packages and their dependencies. The conda command is the primary interface for managing installations of various packages. It can query and search the package index and current installation, create new environments, and install and update packages into existing conda environments. See our Using conda section for more information.

>Conda is also an environment manager application. A conda environment is a directory that contains a specific collection of conda packages that you have installed. For example, you may have one environment with NumPy 1.7 and its dependencies, and another environment with NumPy 1.6 for legacy testing. If you change one environment, your other environments are not affected. You can easily activate or deactivate (switch between) these environments. You can also share your environment with someone by giving them a copy of your environment.yaml file.

Turns out we can even mix in `pip` based installs with our `conda` based packages. We'll likely run into this at some point when we need a package that's not available through the Anaconda Python distro or the Anaconda.org site (like PyPI but for Conda).

# Example 1: Processing data across multiple worksheets
I mentioned in class that I was involved in some research projects in which stream temperature loggers are used to gather temperature data in a bunch of streams throughout the world.

http://advances.sciencemag.org/content/5/1/eaav0486

http://www.geminidataloggers.com/data-loggers/tinytag-plus-2/tgp-4017

https://www.maximintegrated.com/en/products/ibutton/ibuttons/thermochron.cfm

Scientists download the data from the logger into either CSV or Excel files. The data file format can vary. It's a very challenging data cleaning and standardizing problem. For now, we'll consider one specific file type that gets submitted - a single Excel file with each logger's output living in its own sheet within the data file.

The file *sites_1_6.xlsx* contains six sheets named Site1, Site2, ..., Site6. Each sheet contains two columns:

    datetime	            temp_c
    09/15/15 12:00:00 PM	10.22
    09/15/15 01:00:00 PM	10.32
    09/15/15 02:00:00 PM	10.44
    09/15/15 03:00:00 PM	10.49
    09/15/15 04:00:00 PM	10.57
    09/15/15 05:00:00 PM	10.61
    ...

We aren't sure how many rows each file has.

Let's use Python to get the data from each sheet, add a new column with the name of the site, and consolidate all six sheets into a single set of records that we can write out as a CSV file.

Instead of just using `xlrd`, let's also use the newer `openpyxl` library. It appears to be the recommended library for XLSX files (see http://www.python-excel.org/).




### Using `xlrd`
By reading the example covered in Chapter 4 of DWrP and maybe a little Googling, we can figure out how to accomplish our goal using `xlrd`. Let's start by just opening the workbook and printing out the sheet names and the number of rows in each sheet. See p76-77.

In [ ]:
import xlrd

fn_xlsx = 'data/sites_1_6.xlsx'

# Open the workbook
wb_sites = xlrd.open_workbook(fn_xlsx)

# Print out the sheet names and number of rows
for sheet in wb_sites.sheets():
    print(sheet.name, sheet.nrows)


Let's try to read the rows and add the site as a new column.
From the DWwP book (p78-79), we see that each row is returned
as a Python list with column values becoming list elements

In [ ]:
for sheet in wb_sites.sheets():
    print(sheet.name, sheet.nrows)
    site = sheet.name  # Get the site name from the sheet name

    # Before trying to write out all the rows, make
    # sure things are working with just a few rows.
    # Make sure we know if rows are 1 based or 0 based
    print(sheet.row_values(0))
    for rownum in range(1,5):
        print(sheet.row_values(rownum))     # Print the current row
        datarow = sheet.row_values(rownum)  # Store current row in a variable (a list)
        datarow.append(site)                # Append the site name to the datarow list
        print(datarow)                      # Make sure site got appended

Now that we know our approach works, let's accumulate the rows (lists) into another list. We'll end up with a list of lists.

In [ ]:
# Create empty list
data = []

for sheet in wb_sites.sheets():
    print(sheet.name, sheet.nrows)
    site = sheet.name

    # Get the rows for the current sheet
    for rownum in range(1,sheet.nrows): # Think carefully about the sheet.nrows
        datarow = sheet.row_values(rownum)
        datarow.append(site)
        data.append(datarow)

print(len(data))  # Let's confirm that we got all the rows

Finally, how to write this list of lists to a CSV file?
Googled "python write list of lists to csv" and found this:
http://stackoverflow.com/questions/14037540/writing-a-python-list-of-lists-to-a-csv-file. You'll notice that the solution is very similar to what we did when reading from a CSV file using the `csv` library. Not surprising.


In [ ]:
import csv

fn_csv = 'data/sites_1_6_xlrd.csv'

with open(fn_csv, "w") as f:
    writer = csv.writer(f)
    writer.writerows(data)

Now that we've got all the pieces working, let's put it all together as a function. Our function will have two input arguments:

* a string representing the XLSX filename to process
* a string representing the CSV filename to create

Instead of doing this in a notebook, I did it in PyCharm. I wanted to be able to use a debugger and some of the other nice features of an IDE. Start up PyCharm and I'll talk us through getting the correct file open (*reading_excel_temperature_data.py*). I've put this approach together as a function and also redid it using `openpyxl`, an alternative (and newer) library for working with XLSX files. See https://openpyxl.readthedocs.io/en/default/index.html.